In [ ]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 1.8 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=4fcaf6aac4e9325a53c67ccd0c2245c9f7f7667a8dc295ec21fd181a85bea0d8
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [ ]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.5 MB/s 


In [ ]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [ ]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [ ]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0601.csv		       Github
 csv			       heatmap529_UK.ipynb
 csv.zip		       heatmap530.ipynb
 dataframe_csv		       Heat_Map_Online_language.ipynb
 dataframe_csv_EU	       __MACOSX
 dataframe_csv_UK	       non_violant1.csv
 dataframe_result_529_UK.csv   non_violant2.csv
 dataframe_result_530_UK.csv   UK
 dataframe_result_531_UK.csv   violant.csv
 dataframe_result_csv	      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [ ]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [ ]:
ct_path_601 = 'csv/US05_29.csv'

In [ ]:
ct = pd.read_csv(ct_path_601)

In [ ]:
ct.shape[1]

47

In [ ]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [ ]:
#len(columns1)
len(columns2)

47

In [ ]:
ct.columns=columns2

In [ ]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,104380826317363_2958316794257071,Facebook,2020-05-29 22:10:01,2021-06-15 16:21:42,link,Aftershocks of George Floyd's death hit Centra...,theledger.com,George Floyd attended South Florida State Coll...,George Floyd attended South Florida State Coll...,1.0,https://www.theledger.com/news/20200529/afters...,https://www.facebook.com/104380826317363/posts...,3375,-1.250000,1.0,5,7,0,0,2,0,2,0,0,0,3,2,2,2,2,3,2,4,0,1,23719,News Chief,newschief,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The News Chief is a daily newspaper in Winter ...,False,2011-04-27 15:29:39,en,0,23719|2958316794257071
1,2015249512117888_2353472281628941,Facebook,2020-05-29 22:10:00,2021-07-14 01:56:00,link,Justice — Zina for Congress,zinaforcongress.com,Protesters taking to the streets to seek justi...,"Ahmaud Arbery, Breonna Taylor and George Floyd...",2.0,http://www.zinaforcongress.com/justice,https://www.facebook.com/2015249512117888/post...,796,1.826087,1.0,31,6,0,5,0,0,0,0,0,1,6,2,2,3,2,3,2,3,0,2,8581055,Zina Spezakis for Congress NJ9,zspezakisNJ,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,Our leadership is ineffective and corrupt. It ...,False,2019-02-12 20:21:37,en,0,8581055|2353472281628941
2,127784800585461_3342576742439568,Facebook,2020-05-29 22:10:00,2021-07-20 14:56:16,link,A Fearless Response To George Floyd’s Death | ...,fearlessfollower.org,A Fearless Response To George Floyd’s Death by...,Hey Fearless Followers! Here's a challenging r...,1.0,https://fearlessfollower.org/a-fearless-respon...,https://www.facebook.com/127784800585461/posts...,4484,3.263158,1.0,16,29,4,11,0,0,2,0,0,0,3,3,2,3,0,0,0,0,0,1,1467090,Ridge Point Community Church,ridgepointchurch,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Ridge Point exists to Follow Jesus and Fearles...,False,2010-06-14 17:44:07,en,0,1467090|3342576742439568
3,7127779172_10158347173219173,Facebook,2020-05-29 22:09:59,2021-07-19 06:12:54,link,The Shooting Of Black Americans Started Long B...,pridenation.com,By Taryn Finley | HuffPost Donald Trump threat...,The Shooting Of Black Americans Started Long B...,2.0,http://ow.ly/8IiY50zUbyT,https://www.facebook.com/7127779172/posts/1015...,2459,-0.900000,1.0,0,0,0,0,0,0,1,0,0,0,1,2,1,1,1,1,2,1,0,1,8609875,PNN Media Group,pridenation,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,PHARMATIVE PNN Media Group is the parent compa...,False,2008-01-07 06:37:40,en,0,8609875|10158347173219173
4,57218502143_10157744314187144,Facebook,2020-05-29 22:09:54,2021-08-06 00:24:46,link,CN2 Newscast 5/29/20 - CN2 News,cn2.com,A pastor spreading a message of unity and love...,A pastor spreading a message of unity and love...,2.0,https://www.cn2.com/cn2-newscast-5-29-20/,https://www.facebook.com/57218502143/posts/101...,24960,-6.000000,1.0,7,0,0,0,0,0,0,0,0,0,8,6,3,4,4,4,4,9,0,3,1486517,CN2 News,CN2RockHill,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NEWS TIP? 803-326-2762 or news@cn2.com\n\n,False,2009-03-09 22:18:59,en,0,1486517|10157744314187144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23895,203828796317254_3382699755096793,Faceb

In [ ]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [ ]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [ ]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [ ]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [ ]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [ ]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [ ]:
ct['NER-msg'] = ct['message'].apply(ner)

In [ ]:
ct['NER-desc'] = ct['description'].apply(ner)

In [ ]:
ct['NER-title'] = ct['title'].apply(ner)

In [ ]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,"{'Avon Park': 1, 'Polk County': 1, 'Minneapoli...","{'Avon Park': 1, 'Polk County': 1}"
1,{'America': 1},{}
2,{},{}
3,{},{}
4,{},{}


In [ ]:
ct.to_csv('a0601.csv')

In [ ]:
ct=pd.read_csv('a0601.csv')

In [ ]:
!ls

In [ ]:
###
ct=pd.read_csv('a0601.csv')

In [ ]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,104380826317363_2958316794257071,Facebook,2020-05-29 22:10:01,2021-06-15 16:21:42,link,Aftershocks of George Floyd's death hit Centra...,theledger.com,George Floyd attended South Florida State Coll...,George Floyd attended South Florida State Coll...,1.0,https://www.theledger.com/news/20200529/afters...,https://www.facebook.com/104380826317363/posts...,3375,-1.250000,1.0,5,7,0,0,2,0,2,0,0,0,3,2,2,2,2,3,2,4,0,1,23719,News Chief,newschief,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The News Chief is a daily newspaper in Winter ...,False,2011-04-27 15:29:39,en,0,23719|2958316794257071,"{'Avon Park': 1, 'Polk County': 1, 'Minneapoli...","{'Avon Park': 1, 'Polk County': 1}",{}
1,1,2015249512117888_2353472281628941,Facebook,2020-05-29 22:10:00,2021-07-14 01:56:00,link,Justice — Zina for Congress,zinaforcongress.com,Protesters taking to the streets to seek justi...,"Ahmaud Arbery, Breonna Taylor and George Floyd...",2.0,http://www.zinaforcongress.com/justice,https://www.facebook.com/2015249512117888/post...,796,1.826087,1.0,31,6,0,5,0,0,0,0,0,1,6,2,2,3,2,3,2,3,0,2,8581055,Zina Spezakis for Congress NJ9,zspezakisNJ,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,Our leadership is ineffective and corrupt. It ...,False,2019-02-12 20:21:37,en,0,8581055|2353472281628941,{'America': 1},{},{}
2,2,127784800585461_3342576742439568,Facebook,2020-05-29 22:10:00,2021-07-20 14:56:16,link,A Fearless Response To George Floyd’s Death | ...,fearlessfollower.org,A Fearless Response To George Floyd’s Death by...,Hey Fearless Followers! Here's a challenging r...,1.0,https://fearlessfollower.org/a-fearless-respon...,https://www.facebook.com/127784800585461/posts...,4484,3.263158,1.0,16,29,4,11,0,0,2,0,0,0,3,3,2,3,0,0,0,0,0,1,1467090,Ridge Point Community Church,ridgepointchurch,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Ridge Point exists to Follow Jesus and Fearles...,False,2010-06-14 17:44:07,en,0,1467090|3342576742439568,{},{},{}
3,3,7127779172_10158347173219173,Facebook,2020-05-29 22:09:59,2021-07-19 06:12:54,link,The Shooting Of Black Americans Started Long B...,pridenation.com,By Taryn Finley | HuffPost Donald Trump threat...,The Shooting Of Black Americans Started Long B...,2.0,http://ow.ly/8IiY50zUbyT,https://www.facebook.com/7127779172/posts/1015...,2459,-0.900000,1.0,0,0,0,0,0,0,1,0,0,0,1,2,1,1,1,1,2,1,0,1,8609875,PNN Media Group,pridenation,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,PHARMATIVE PNN Media Group is the parent compa...,False,2008-01-07 06:37:40,en,0,8609875|10158347173219173,{},{},{}
4,4,57218502143_10157744314187144,Facebook,2020-05-29 22:09:54,2021-08-06 00:24:46,link,CN2 Newscast 5/29/20 - CN2 News,cn2.com,A pastor spreading a message of unity and love...,A pastor spreading a message of unity and love...,2.0,https://www.cn2.com/cn2-newscast-5-29-20/,https://www.facebook.com/57218502143/posts/101...,24960,-6.000000,1.0,7,0,0,0,0,0,0,0,0,0,8,6,3,4,4,4,4,9,0,3,1486517,CN2 News,CN2RockHill,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NEWS TIP? 803-326-2762 or news@cn2.com\n\n,False,2009-03-09 22:18:59,en,0,1486517|10157744314187144,{},{},{}
...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [ ]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [ ]:
###
ct_ner = ct

In [ ]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,"{'Avon Park': 1, 'Polk County': 1, 'Minneapoli...","{'Avon Park': 1, 'Polk County': 1}",{}
1,{'America': 1},{},{}
2,{},{},{}
3,{},{},{}
4,{},{},{}


In [ ]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [ ]:
ct_ner['NER-msg']

0        [Minneapolis, Polk County, Avon Park]
1                                    [America]
2                                           []
3                                           []
4                                           []
                         ...                  
23895                            [Minneapolis]
23896                                     [US]
23897           [Minneapolis, Hennepin County]
23898                                       []
23899                                       []
Name: NER-msg, Length: 23900, dtype: object

In [ ]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [ ]:
ct_ner['NER']

0        [Minneapolis, Polk County, Avon Park, Polk Cou...
1                                                [America]
2                                                       []
3                                                       []
4                                                       []
                               ...                        
23895              [Minneapolis, Minneapolis, Minneapolis]
23896                       [US, Minneapolis, Minneapolis]
23897    [Minneapolis, Hennepin County, Minneapolis, Mi...
23898                                                   []
23899                                        [Minneapolis]
Name: NER, Length: 23900, dtype: object

In [ ]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [ ]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [ ]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,104380826317363_2958316794257071,Facebook,2020-05-29 22:10:01,2021-06-15 16:21:42,link,Aftershocks of George Floyd's death hit Centra...,theledger.com,George Floyd attended South Florida State Coll...,George Floyd attended South Florida State Coll...,1.0,https://www.theledger.com/news/20200529/afters...,https://www.facebook.com/104380826317363/posts...,3375,-1.250000,1.0,5,7,0,0,2,0,2,0,0,0,3,2,2,2,2,3,2,4,0,1,23719,News Chief,newschief,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The News Chief is a daily newspaper in Winter ...,False,2011-04-27 15:29:39,en,0,23719|2958316794257071,"[Minneapolis, Polk County, Avon Park]","[Polk County, Avon Park]",[],"[minneapolis, polk county, avon park]"
1,1,2015249512117888_2353472281628941,Facebook,2020-05-29 22:10:00,2021-07-14 01:56:00,link,Justice — Zina for Congress,zinaforcongress.com,Protesters taking to the streets to seek justi...,"Ahmaud Arbery, Breonna Taylor and George Floyd...",2.0,http://www.zinaforcongress.com/justice,https://www.facebook.com/2015249512117888/post...,796,1.826087,1.0,31,6,0,5,0,0,0,0,0,1,6,2,2,3,2,3,2,3,0,2,8581055,Zina Spezakis for Congress NJ9,zspezakisNJ,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,Our leadership is ineffective and corrupt. It ...,False,2019-02-12 20:21:37,en,0,8581055|2353472281628941,[America],[],[],[america]
2,2,127784800585461_3342576742439568,Facebook,2020-05-29 22:10:00,2021-07-20 14:56:16,link,A Fearless Response To George Floyd’s Death | ...,fearlessfollower.org,A Fearless Response To George Floyd’s Death by...,Hey Fearless Followers! Here's a challenging r...,1.0,https://fearlessfollower.org/a-fearless-respon...,https://www.facebook.com/127784800585461/posts...,4484,3.263158,1.0,16,29,4,11,0,0,2,0,0,0,3,3,2,3,0,0,0,0,0,1,1467090,Ridge Point Community Church,ridgepointchurch,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Ridge Point exists to Follow Jesus and Fearles...,False,2010-06-14 17:44:07,en,0,1467090|3342576742439568,[],[],[],[]
3,3,7127779172_10158347173219173,Facebook,2020-05-29 22:09:59,2021-07-19 06:12:54,link,The Shooting Of Black Americans Started Long B...,pridenation.com,By Taryn Finley | HuffPost Donald Trump threat...,The Shooting Of Black Americans Started Long B...,2.0,http://ow.ly/8IiY50zUbyT,https://www.facebook.com/7127779172/posts/1015...,2459,-0.900000,1.0,0,0,0,0,0,0,1,0,0,0,1,2,1,1,1,1,2,1,0,1,8609875,PNN Media Group,pridenation,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,PHARMATIVE PNN Media Group is the parent compa...,False,2008-01-07 06:37:40,en,0,8609875|10158347173219173,[],[],[],[]
4,4,57218502143_10157744314187144,Facebook,2020-05-29 22:09:54,2021-08-06 00:24:46,link,CN2 Newscast 5/29/20 - CN2 News,cn2.com,A pastor spreading a message of unity and love...,A pastor spreading a message of unity and love...,2.0,https://www.cn2.com/cn2-newscast-5-29-20/,https://www.facebook.com/57218502143/posts/101...,24960,-6.000000,1.0,7,0,0,0,0,0,0,0,0,0,8,6,3,4,4,4,4,9,0,3,1486517,CN2 News,CN2RockHill,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NEWS TIP? 803-326-2762 or news@cn2.com\n\n,False,2009-03-09 22:18:59,en,0,1486517|10157744314187144,[],[],[],[]
...,...,...,

In [ ]:
ct_ner['NER-1']=ct_ner['NER']

In [ ]:
ct_ner['NER-1'][0]

['minneapolis', 'polk county', 'avon park']

In [ ]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 8.3 MB/s 


In [ ]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [ ]:
list=[]
for i in ct_ner['NER-1'][0]:
  geolocator = Nominatim(user_agent="address_development")
  location = geolocator.geocode(i,timeout=30)
  list.append(location.address)
ct_ner['NER-1'][0]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
ct_ner['NER-1'][0]

['Minneapolis, Hennepin County, Minnesota, 55415:55454, United States',
 'Polk County, Texas, United States',
 'Avon Park, Highlands County, Florida, 33825, United States']

In [ ]:
ct_ner['NER-1'][0]

['minneapolis', 'polk county', 'avon park']

In [ ]:
ner_lst = ct_ner['NER'].values

In [ ]:
ner_lst

array([list(['the city of new orleans', 'new orleans']),
       list(['mississippi', 'the united states', 'jackson', 'miss.']),
       list(['mississippi', 'the united states', 'jackson', 'miss.']),
       ..., list(['minneapolis']),
       list(['new mexico', 'albuquerque', 'albuquerque', 'new mexico’s']),
       list(['new mexico', 'oakland county', 'minneapolis', 'oakland', 'west bloomfield'])],
      dtype=object)

In [ ]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [ ]:
list

['the city of new orleans',
 'new orleans',
 'mississippi',
 'the united states',
 'jackson',
 'miss.',
 'mississippi',
 'the united states',
 'jackson',
 'miss.',
 'mississippi',
 'the united states',
 'jackson',
 'miss.',
 'minneapolis',
 'memphis',
 'minneapolis',
 'the united states',
 'the village of bath police department',
 'bath new york',
 'bath n.y.',
 'ny',
 'bath new york',
 'ny',
 'sacramento',
 'the village of bath police department',
 'sacramento county',
 'harrisonburg',
 'bath new york',
 'harrisonburg',
 'bath new york',
 'spartanburg',
 'minneapolis',
 'bath new york',
 'the city of riverside',
 'minnesota',
 'riverside',
 'minnesota',
 'minnesota',
 'minnesota',
 'dekalb',
 'washington',
 'd.c.',
 'minneapolis',
 'illinois',
 'u.s.',
 'michigan',
 'michigan',
 'washington',
 'd.c.',
 'america',
 'minneapolis',
 'washington',
 'minneapolis',
 'america',
 'minneapolis',
 'america',
 'minneapolis',
 'santa rosa',
 'alameda county',
 'philadelphia',
 'pennsylvania',
 'r

In [ ]:
c = collections.Counter()

In [ ]:
dict_a=collections.Counter(list).most_common()

In [ ]:
dict_a

[('minneapolis', 8856),
 ('minnesota', 2579),
 ('america', 2183),
 ('u.s.', 1792),
 ('the united states', 1658),
 ('us', 1291),
 ('washington', 639),
 ('chicago', 551),
 ('california', 439),
 ('atlanta', 438),
 ('los angeles', 438),
 ('texas', 396),
 ('new york', 368),
 ('d.c.', 362),
 ('flint', 353),
 ('houston', 352),
 ('boston', 337),
 ('michigan', 332),
 ('seattle', 319),
 ('new york city', 313),
 ('dc', 304),
 ('london', 263),
 ('florida', 261),
 ('philadelphia', 252),
 ('portland', 237),
 ('nyc', 232),
 ('georgia', 229),
 ('camden', 227),
 ('oakland', 224),
 ('nj', 186),
 ('hennepin county', 183),
 ('sacramento', 179),
 ('denver', 170),
 ('los angeles county', 169),
 ('louisville', 167),
 ('connecticut', 163),
 ('st. paul', 157),
 ('dallas', 157),
 ('mn', 150),
 ('brooklyn', 143),
 ('baltimore', 143),
 ('kentucky', 143),
 ('la county', 139),
 ('maine', 138),
 ('milwaukee', 135),
 ('manhattan', 135),
 ('san francisco', 134),
 ('wisconsin', 131),
 ('detroit', 129),
 ('santa monica'

In [ ]:
dataframe601=pd.DataFrame(dict_a)

In [ ]:
len(dict_a)

3063

In [ ]:
dataframe601.to_csv('dataframe0601.csv')